In [1]:
import glob
import json
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip "/content/drive/MyDrive/032.특허 분야 자동분류 데이터.zip" -d "dataset"

Archive:  /content/drive/MyDrive/032.특허 분야 자동분류 데이터.zip
   creating: dataset/01.데이터/
   creating: dataset/01.데이터/1.Training/
   creating: dataset/01.데이터/1.Training/라벨링데이터/
  inflating: dataset/01.데이터/1.Training/라벨링데이터/TL1.zip  
   creating: dataset/01.데이터/1.Training/원천데이터/
  inflating: dataset/01.데이터/1.Training/원천데이터/TS1.zip  
   creating: dataset/01.데이터/2.Validation/
   creating: dataset/01.데이터/2.Validation/라벨링데이터/
  inflating: dataset/01.데이터/2.Validation/라벨링데이터/VL1.zip  
   creating: dataset/01.데이터/2.Validation/원천데이터/
  inflating: dataset/01.데이터/2.Validation/원천데이터/VS1.zip  


In [4]:
%cd dataset

/content/dataset


In [5]:
!find . -name "*.zip" | while read filename; do unzip -o -d "`basename -s .zip "$filename"`" "$filename"; done;

Archive:  ./01.데이터/1.Training/원천데이터/TS1.zip
   creating: TS1/A_농업_임업및어업_01_03/
   creating: TS1/A_농업_임업및어업_01_03/01_농업/
   creating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01110_곡물및기타식량작물재배업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01121_채소작물재배업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01122_화훼작물재배업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01123_종자및묘목생산업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01131_과실작물재배업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01132_음료용및향신용작물재배업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01140_기타작물재배업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01151_콩나물재배업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/011_작물재배업/01159_기타시설작물재배업.json  
   creating: TS1/A_농업_임업및어업_01_03/01_농업/012_축산업/
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/012_축산업/01211_젖소사육업.json  
  inflating: TS1/A_농업_임업및어업_01_03/01_농업/012_축산업/01212_육우사육업.js

In [6]:
def read_dataset(path):
    file_list = glob.glob(path, recursive=True)
    dfs = []
    for file in file_list:
        with open(file) as f:
            json_data = pd.json_normalize(json.loads(f.read())['dataset'])
        dfs.append(json_data)
    return pd.concat(dfs, sort=False)

In [7]:
def get_dataset(data_type):
    source_df = read_dataset(f'{data_type}S1/**/*.json')
    label_df = read_dataset(f'{data_type}L1/**/*.json')

    source_df = source_df[['documentId', 'invention_title', 'abstract', 'claims']].drop_duplicates(subset=['documentId'])
    label_df = label_df[['documentId', 'LLno', 'Lno', 'Mno', 'Sno', 'SSno']].drop_duplicates(subset=['documentId'])

    df = pd.merge(source_df, label_df, on='documentId')
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)

    return df

In [8]:
train_df = get_dataset('T')
val_df = get_dataset('V')

In [9]:
train_df.head()

,documentId,invention_title,abstract,claims,LLno,Lno,Mno,Sno,SSno
0,kr19930012899u,발효톱밥과 배합사료를 혼합하여 급여하는 자동장치,본 고안은 축산업의 발전과 변화에 따라서 환경오염을 막아주고 악취를 제거하며 가축의...,1. 본 고안은 축산업의 사료급여의 변화에 따라서 발효톱밥을 사료와 혼합하여 급여하...,A,01,012,0123,01239
1,kr19930014426u,학습용 이층 새장,본 고안은 학습용 이층 새장으로서 기대에 사방을 유리로 접합시키고 새장안을 이층을 ...,학습용 이층 새장 사방이 유리로 접합되고 새장 상층 바닥(3)에 상층 바닥 물구멍(...,A,01,012,0123,01239
2,kr19940005237u,동물사료 자동공급장치,본 고안은 동물사료 자동공급장치에 관한 것이다. 종래에는 관리자가 수족관이나 새장등...,"1. 상, 하부에 사료투입구(11)와 하단개구부(12)가 천공된 본체케이스(10)와...",A,01,012,0123,01239
3,kr19950005558u,이동식 사료 혼합기,"본 고안은 젖소 등의 가축을 사육함에 있어서, 조사료를 비롯한 각종 사료를 효과적으...",트레일러상의 혼합통 본체(1)의 전면에 장설된 기어 박스(2)의 동력에 의해 내부에...,A,01,012,0123,01239
4,kr19950013738u,축사용 바닥재,"본 고안은 축사용 바닥재에 관한 것으로 특히, 축사의 바닥에 설치하여서 소나 돼지등...","1. 축사용 바닥재를 구성함에 있어서, 성광하협의 모양을 갖는 결합공을 상측에 일정...",A,01,012,0123,01239


In [10]:
val_df.head()

,documentId,invention_title,abstract,claims,LLno,Lno,Mno,Sno,SSno
0,kr20160183401b1,락토바실러스 균주 조합을 이용한 반추동물용 발효 완전배합사료,"본 발명은 반추동물 전용 섬유질 완전배합사료(total mixed ration, T...",섬유질 완전배합사료를 락토바실러스 파라카제이(Lactobacillus paracas...,A,01,012,0123,01239
1,kr20160183836b1,메탄저감용 사료 첨가제,"본 발명은 사료 첨가제에 관한 것으로, 글루타민산(Grutamic acid) 10~...","글루타민산(Glutamic acid) 10~20중량%, 글루콘산 아연 30~65중량...",A,01,012,0123,01239
2,kr20167015383b1,동물 사료 조정을 위한 시스템 및 방법,본 출원은 동물 사료 성분의 소화율을 개선하기 위해 동물 사료를 분석하고 동물 사료...,"하기 단계를 포함하는, 돼지 또는 가금류의 사료의 조정된 조성물을 선택하기 위하여 ...",A,01,012,0123,01239
3,kr20167019304a,"애완동물 사료용 기호성 증진제, 그의 제조 방법 및 용도","본 발명은 애완동물 사료, 바람직하게 개 사료에 사용하기 위한 아스코르브산; 그의 ...",아스코르브산; 그의 이성체; 그의 유도체; 그의 염; 및 그의 조합으로 구성된 그룹...,A,01,012,0123,01239
4,kr20167023800a,동물 사료,여기에 개시된 것은 가축을 먹이기 위한 고형 사료 제제이다. 제제는 단백질 성분으로...,"가축에게 먹이기 위한 고형 사료 제제로, 상기 제제는 단백질 성분으로서 분쇄된 솔트...",A,01,012,0123,01239


In [11]:
print(f'Train: {len(train_df)}')
print(f'Validation: {len(val_df)}')

Train: 379316
Validation: 51071


In [12]:
train_df.to_csv('/content/drive/MyDrive/train.csv')
val_df.to_csv('/content/drive/MyDrive/val.csv')